# 4) Multilayer Network

## a) Dense multilayer neural network architecture

### Theory

To obtain a deep learning neural network, all you need to do is add layers. The number of layers in the network, denoted $D$, is called the depth of the network. Each layer is given an index $k$ and contains a weight matrix $W^{(k)}$ and a bias vector $b^{(k)}$. The $k$-th layer will have as input values the output values of the previous layer. We will only consider dense networks, i.e. neural network where each neuron of a layer is connected to all the output values of the previous layer. The weight matrix and bias vectors will therefore be of the form

\begin{equation}
    W^{(k)} = \left( \begin{array}{ccc}
        W_{11} & W_{12} & \cdots & W_{1 N_{k-1}} \\
        W_{21} & W_{22} & \cdots & W_{2 N_1} \\
        \vdots & \ddots & \ddots & \vdots\\
        W_{N_k 1} & W_{N_0 2} & \cdots & W_{N_k N_{k-1}}\end{array} \right)\qquad \qquad \qquad
     \mathbf{b}^{(k)} = \left( \begin{array}{ccc}
        b_1 \\
        b_2 \\
        \vdots \\
        b_{N_k} \end{array} \right)
\end{equation}

Before and after applying the non-linear function $\sigma^{(k)}$ associated with the $k$-th layer, the $i$-th output of the $k$-th layer will be respectively

\begin{equation}
    z_i^{(k)} = \left(W^{(k)} \mathbf{y}^{(k-1)} + \mathbf{b}^{(k)}\right)_i = W_{i1}^{(k)} y_1^{(k-1)} + W_{i2}^{(k)} y_2^{(k-1)} + \cdots + W_{iN_0}^{(k)} y_{N_0}^{(k-1)} + b_i^{(k)}
\end{equation}

and

\begin{equation}
    y_i^{(k)} = \sigma^{(k)} \left[\left(W^{(k)} \mathbf{y}^{(k-1)} + \mathbf{b}^{(k)}\right)_i \right] = \sigma^{(k)} \left(W_{i1}^{(k)} y_1^{(k-1)} + W_{i2}^{(k)} y_2^{(k-1)} + \cdots + W_{iN_0}^{(k)} y_{N_0}^{(k-1)} + b_i^{(k)}\right)
\end{equation}

where $y_{j}^{(k-1)}$ is the output of the previous layer, with $\mathbf{y}^{(0)} = \mathbf{x}$. At the very end of the network, the $N_D$ output values are given by

\begin{equation}
    y_i^{(D)} = \sigma^{(D)} \left(W_{i1}^{(D)} y_1^{(D-1)} + W_{i2}^{(D)} y_2^{(D-1)} + \cdots + W_{iN_0}^{(D)} y_{N_0}^{(D-1)} + b_i^{(D)}\right)
\end{equation}

and we can then calculate the difference between the network output and the expected output and define an error

\begin{equation}
    \delta_i = y_i^{(D)} - \xi_i
\end{equation}

\begin{equation}
    \epsilon_i = \left(y_i^{(D)} - \xi_i\right)^2.
\end{equation}

The total error being the sum of the individual errors

\begin{equation}
    \epsilon = \sum_{i=1}^{N_1}\left(y_i^{(D)} - \xi_i\right)^2.
\end{equation}

### Julia code

Let's start by defining the non-linear function that we will give to our different layers. This function doesn't necessarily have to be the same in each layer, although that's what we'll do here.

In [1]:
function ReLu(p::Float64)::Float64
    if p > 0
        return p
    else
        return 0.0
    end
end

ReLu (generic function with 1 method)

Let's now define a single layer as we did earlier.

In [2]:
mutable struct SingleLayer
    W::Array{Float64,2}
    b::Vector{Float64}
    σ::Function
end

(l::SingleLayer)(x::Array{Float64,1}) = l.σ.(l.W * x + l.b)

A multilayer network is made up of a set of single layers. We therefore store the single layers in a vector in the multilayer structure. Next, we create a function on the multilayer structure that will pass a vector of input data through the entire network. This is based on the function defined on the single layer structure.

In [3]:
mutable struct MultiLayerModel
    Layers::Vector{SingleLayer}
end

function (m::MultiLayerModel)(x::Array{Float64,1})
    y = x
    
    for i in 1:length(m.Layers)
        y = m.Layers[i](y)
    end
    
    return y
end

Finally, to simplify the creation of a multilayer network, we define two functions: Dense and Chain. The Dense function creates a single layer based on the number of inputs and outputs in that layer and its non-linear function. If no non-linear function is specified, the identity function will simply be applied. The Chain function takes a certain number of simple layers as arguments and generates the multi-layer network corresponding to the order of the layers given as arguments.

In [4]:
function Dense(nInput::Int64,nOutput::Int64, σ::Function=identity)
    return SingleLayer(rand(nOutput,nInput), rand(nOutput), σ)
end

function Chain(SingleLayers::SingleLayer...)
    Layers = Vector{SingleLayer}(undef,length(SingleLayers))
    for k in 1:length(SingleLayers)
        Layers[k] = SingleLayers[k]
    end
    return MultiLayerModel(Layers)
end

Chain (generic function with 1 method)

We now instantiate the multilayer network using the Chain and Dense functions. We can now pass an input vector through the network to obtain its final outputs and the associated error.

In [12]:
n0 = 5;
n1 = 8;
n2 = 7;
n3 = 3;

#Créé un réseau à trois couches avec respectivement n1, n2 et n3 neurones. La fonction non-linéaire est ReLu
#sur les deux premières couches et on ne met rien sur la troisième.
model = Chain(Dense(n0,n1, ReLu),Dense(n1,n2, ReLu),Dense(n2,n3))

y0 = rand(n0)
ξ = rand(n3)

yD = model(y0)
δ = yD-ξ
ϵ = δ.^2
ϵTot = sum(ϵ)
println(yD)
println(δ)
println(ϵ)
println(ϵTot)

[49.195426458436515, 47.90973784360255, 28.49225628130916]
[48.19634544937155, 47.284638318825074, 27.610533821848247]
[2322.8877146751574, 2235.8370209421005, 762.341577927426]
5321.066313544683


## b) Back-propagation algorithm

### Theory

An historical problem with multilayer networks has been to find a way of training them correctly and efficiently (numerically speaking). The $\epsilon$ error explicitly contains only the weights of the last layer and not those of the previous layers. We can therefore use the method found for the single layer to update the weights of the $D$ layer via

\begin{equation}
    \left(W_{ij}^{(D)}\right)' = W_{ij}^{(D)} - \alpha y_j^{(D-1)} \delta_i.
\end{equation}

However, this method cannot be used to update the weights of previous layers. In fact, what is the $\delta_i$ difference that would allow the weights to be pushed in the right direction? We therefore need to find a correctly defined $\delta^{(k)}$ for each $k$-th layer.

To do this, let's start as always with the derivative of the error as a function of a certain weight in a certain layer

\begin{equation}
    \frac{\partial \epsilon}{\partial W_{ij}^{(k)}} = 2 \sum_{l=1}^{N_D} \left( y_l^{(D)} - \xi_l \right)\frac{\partial y_l^{(D)}}{\partial W_{ij}^{(k)}}.
\end{equation}

Using the derivation theorem for compound functions, we obtain

\begin{equation}
    \frac{\partial y_l^{(D)}}{\partial W_{ij}^{(k)}} = \left( \sigma^{(D)} \right)' \left( z_l^{(D)}\right)\frac{\partial z_l^{(D)}}{\partial W_{ij}^{(k)}}
\end{equation}

where $z_l^{(D)} = \sum_{m=1}^{N_{D-1}} W_{lm}^{(D)} y_m^{(D-1)}+b_m^{(D)}$. If $k=D$, then we obtain the usual formula with $y_j^{(D-1)}\delta_i$. On the other hand, if $k<D$, we need to push the derivation further.

\begin{equation}
    \frac{\partial z_l^{(D)}}{\partial W_{ij}^{(k)}} = \sum_{m=1}^{N_{D-1}} W_{lm}^{(D)} \frac{\partial y_m^{(D-1)}}{\partial W_{ij}^{(k)}}.
\end{equation}

Here we can see a pattern taking shape as, starting from the derivative $\frac{\partial y_l^{(D)}}{\partial W_{ij}^{(k)}}$, we arrive at the derivative in the previous layer $\frac{\partial y_m^{(D-1)}}{\partial W_{ij}^{(k)}}$. To make this pattern clear, for any layer $k'>k$ we can write

\begin{equation}
    \frac{\partial z_l^{(k')}}{\partial W_{ij}^{(k)}} = \sum_{m=1}^{N_{k'-1}} W_{lm}^{(k')} \left( \sigma^{(k'-1)} \right)' \left( z_m^{(k'-1)}\right) \frac{\partial z_m^{(k'-1)}}{\partial W_{ij}^{(k)}}.
\end{equation}

If we define at each layer $k'>1$ the matrix $\mathbf{M}^{(k’)}$ whose elements are

\begin{equation}
    \mathbf{M}^{(k')}_{lm} = W_{lm}^{(k')}\left( \sigma^{(k'-1)} \right)' \left( z_m^{(k'-1)}\right)
\end{equation}

we can see that the above equation can be written as a matrix product

\begin{equation}
    \frac{\partial z_l^{(k')}}{\partial W_{ij}^{(k)}} = \left[ \mathbf{M}^{(k')} \frac{\partial \mathbf{z}^{(k'-1)}}{\partial W_{ij}^{(k)}} \right]_l.
\end{equation}

Finally, we obtain the formula for the back-propagation algorithm using this matrix product of $\mathbf{M}^{(k')}$ on each layer

\begin{equation}
    \frac{\partial z_l^{(D)}}{\partial W_{ij}^{(k)}} = \left[ \mathbf{M}^{(D)}\mathbf{M}^{(D-1)} \cdots \mathbf{M}^{(k+2)}\mathbf{M}^{(k+1)} \frac{\partial \mathbf{z}^{(k)}}{\partial W_{ij}^{(k)}}\right]_l.
\end{equation}

In this way, we arrive at the correct layer for calculating the derivative of $\mathbf{z}$ as a function of its explicit weights

\begin{equation}
    \frac{\partial z_l^{(k)}}{\partial W_{ij}^{(k)}} = y_j^{(k-1)} \delta_{il}
\end{equation}

### Implementation

The back-propagation algorithm takes the following form

1. Forward Propagation : Pass the input $y^{(0)}$ through the neural network. For each layer, calculate the associated $\mathbf{M}$ matrix.

2. Once the end of the network has been reached, calculate the initial deviation vector

    \begin{equation}
        \Delta_l^{(D)} = \left(  y_l^{(D)} - \xi_l \right) \left( \sigma^{(D)}\right)'\left( z_l^{(D)}\right)
    \end{equation}
    
3. Backward Propagation : For $k$ from $D$ to $1$, update the weights using the formula

    \begin{equation}
        \left(W_{ij}^{(k)}\right)' = W_{ij}^{(k)} - \alpha\Delta_i^{(k)}y_j^{(k-1)}
    \end{equation}

    If $k>1$, update the deviation vector

    \begin{equation}
        \Delta_l^{(k-1)} = \sum_{l'=1}^{N_{k}} \Delta_{l'}^{(k)} M_{l'l}^{(k)}
    \end{equation}
    
Once the algorithm had been completed, all the weights in all the layers were updated.

### Julia code

We define the function for training the multilayer network defined previously on the basis of the back-propagation algorithm. Be careful with the indices, as yMat[1] corresponds to $y^{(0)}$ and M[k] corresponds to $\mathbf{M}^{(k+1)}$. Since the last layer has no non-linear function, $\Delta^{(D)}=\delta$.

In [6]:
function train!(model::MultiLayerModel, x::Array{Float64,1}, ξ::Array{Float64,1}, iteration::Int64, α::Float64)
    sumϵVec = Vector{Float64}(undef,0)
    D = length(model.Layers)
    zMat = Vector{Vector{Float64}}(undef, D)
    yMat = Vector{Vector{Float64}}(undef, D+1)
    yMat[1] = x
    M = Vector{Array{Float64,2}}(undef, D-1)

    for χ in 1:iteration
        
        #Forward propagation
        for k in 1:D
            
            zMat[k] = model.Layers[k].W * yMat[k] + model.Layers[k].b
            yMat[k+1] = model.Layers[k].σ.(zMat[k])

            if k < D
                Mk = Array{Float64,2}(undef, size(model.Layers[k+1].W)[1], size(model.Layers[k+1].W)[2])
                for j in 1:size(model.Layers[k+1].W)[2]
                    for i in 1:size(model.Layers[k+1].W)[1]
                        if zMat[k][j] > 0
                            Mk[i,j] = model.Layers[k+1].W[i,j]
                        else
                            Mk[i,j] = 0
                        end
                    end
                end
                M[k] = Mk
            end
        end
        
        Δ = transpose(yMat[D+1]-ξ)
        ϵ = Δ.^2
        @show sum(ϵ)
        push!(sumϵVec, sum(ϵ))

        #Backward Propagation
        for k in D:-1:1
            for j in 1:size(model.Layers[k].W)[2]
                for i in 1:size(model.Layers[k].W)[1]
                    model.Layers[k].W[i,j] -= α*Δ[i]*yMat[k][j]
                    model.Layers[k].b[i] -= α*Δ[i]
                end
            end
            if k > 1
                Δ = Δ*M[k-1]
            end
        end
    end
    return sumϵVec
end

train! (generic function with 1 method)

In [13]:
sumϵVec = train!(model, y0, ξ, 30 , 0.1);

sum(ϵ) = 5321.066313544683
sum(ϵ) = 2603.037824258862
sum(ϵ) = 234.27340418329754
sum(ϵ) = 21.084606376496794
sum(ϵ) = 1.8976145738847108
sum(ϵ) = 0.1707853116496239
sum(ϵ) = 0.015370678048466167
sum(ϵ) = 0.0013833610243619468
sum(ϵ) = 0.00012450249219258128
sum(ϵ) = 1.1205224297333643e-5
sum(ϵ) = 1.0084701867603453e-6
sum(ϵ) = 9.076231680833754e-8
sum(ϵ) = 8.168608512803568e-9
sum(ϵ) = 7.35174766139525e-10
sum(ϵ) = 6.61657289581038e-11
sum(ϵ) = 5.954915607047867e-12
sum(ϵ) = 5.359424045009763e-13
sum(ϵ) = 4.8234816269188204e-14
sum(ϵ) = 4.3411334710598604e-15
sum(ϵ) = 3.907020082314531e-16
sum(ϵ) = 3.516318460471024e-17
sum(ϵ) = 3.1646865205514392e-18
sum(ϵ) = 2.848215507772521e-19
sum(ϵ) = 2.563404389284905e-20
sum(ϵ) = 2.3070762599048053e-21
sum(ϵ) = 2.0763006326880429e-22
sum(ϵ) = 1.8684003129084435e-23
sum(ϵ) = 1.682042970567843e-24
sum(ϵ) = 1.5152881536554053e-25
sum(ϵ) = 1.3584887367149536e-26


Congratulations, you now know how a neural network works ! 🥳